In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
df = pd.read_csv('dataset/train.csv')

In [3]:
df['text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [4]:
# create a function that will extract all the unique words from the dataset
def unique_words(df):
    unique_words = []
    for i in range(len(df)):
        for word in df['text'][i].split():
            if word not in unique_words:
                unique_words.append(word)
    return unique_words


In [5]:
arr = unique_words(df)

In [6]:
len(arr)

31924

In [7]:
# create a function that will create a dictionary of all the unique words and their frequency in the dataset
def word_freq(df):
    unique_words = []
    for i in range(len(df)):
        for word in df['text'][i].split():
            if word not in unique_words:
                unique_words.append(word)
    word_freq = {}
    for word in unique_words:
        word_freq[word] = 0
    for i in range(len(df)):
        for word in df['text'][i].split():
            word_freq[word] += 1
    return word_freq

In [8]:
word_dict = word_freq(df)

In [9]:
print(word_dict)

{'Our': 25, 'Deeds': 1, 'are': 345, 'the': 2575, 'Reason': 5, 'of': 1722, 'this': 335, '#earthquake': 8, 'May': 16, 'ALLAH': 1, 'Forgive': 1, 'us': 53, 'all': 178, 'Forest': 29, 'fire': 148, 'near': 50, 'La': 11, 'Ronge': 1, 'Sask.': 1, 'Canada': 4, 'All': 52, 'residents': 4, 'asked': 9, 'to': 1805, "'shelter": 1, 'in': 1757, "place'": 1, 'being': 67, 'notified': 1, 'by': 469, 'officers.': 1, 'No': 53, 'other': 31, 'evacuation': 32, 'or': 181, 'shelter': 4, 'place': 22, 'orders': 11, 'expected': 11, '13,000': 1, 'people': 146, 'receive': 2, '#wildfires': 5, 'California': 89, 'Just': 48, 'got': 103, 'sent': 12, 'photo': 8, 'from': 372, 'Ruby': 1, '#Alaska': 2, 'as': 282, 'smoke': 33, 'pours': 1, 'into': 162, 'a': 1845, 'school': 47, '#RockyFire': 4, 'Update': 6, '=>': 1, 'Hwy.': 4, '20': 13, 'closed': 15, 'both': 20, 'directions': 1, 'due': 27, 'Lake': 8, 'County': 30, '-': 763, '#CAfire': 1, '#flood': 3, '#disaster': 5, 'Heavy': 7, 'rain': 27, 'causes': 10, 'flash': 14, 'flooding': 26,

In [10]:
# do tokenization of all the words present in the dictionary
def tokenize_words(dictionary):
    tokens = []
    for word in dictionary:
        tokens.append(word_tokenize(word))
        print(word)